In [5]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 16.5011,
	"longitude": 80.625,
	"daily": "precipitation_sum",
	"timezone": "GMT",
	"start_date": "2025-01-01",
	"end_date": "2025-01-30"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_precipitation_sum = daily.Variables(0).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

daily_data["precipitation"] = daily_precipitation_sum

daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)

file_name = 'precipitation_25'
daily_dataframe['day'] = daily_dataframe['date'].dt.day
daily_dataframe['month'] = daily_dataframe['date'].dt.month
daily_dataframe['year'] = daily_dataframe['date'].dt.year
daily_dataframe['date'] = daily_dataframe['date'].dt.date

daily_dataframe.to_csv(f'./Dataset/Rainfall/Processed_Rainfall/{file_name}.csv', index=None)
print(f"Combined data saved to 'Dataset/Rainfall/Processed_Rainfall/{file_name}.csv'")

Coordinates 16.5°N 80.625°E
Elevation 21.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                        date  precipitation_sum
0  2025-01-01 00:00:00+00:00                0.0
1  2025-01-02 00:00:00+00:00                0.0
2  2025-01-03 00:00:00+00:00                0.0
3  2025-01-04 00:00:00+00:00                0.0
4  2025-01-05 00:00:00+00:00                0.0
5  2025-01-06 00:00:00+00:00                0.0
6  2025-01-07 00:00:00+00:00                0.0
7  2025-01-08 00:00:00+00:00                0.0
8  2025-01-09 00:00:00+00:00                0.0
9  2025-01-10 00:00:00+00:00                0.0
10 2025-01-11 00:00:00+00:00                0.0
11 2025-01-12 00:00:00+00:00                0.0
12 2025-01-13 00:00:00+00:00                0.0
13 2025-01-14 00:00:00+00:00                0.0
14 2025-01-15 00:00:00+00:00                0.0
15 2025-01-16 00:00:00+00:00                0.0
16 2025-01-17 00:00:00+00:00                0.0
17 2025-01-18 00:00:00+00:00       